In [1]:
import pandas as pd
import numpy as np
import struct
import codecs
import matplotlib.pyplot as plt
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number
from scipy import signal
import h5py
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import load_model
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.externals import joblib
import xgboost as xgb
import gcforest.gcforest
from gcforest.utils.config_utils import load_json
from phase_utils import print_cm
from phase_features_loader import PhaseFeaturesLoader
from imblearn.metrics import classification_report_imbalanced
from imblearn.combine import SMOTETomek, SMOTEENN

# Enable inline plotting
%matplotlib inline

/home/cahya/.virtualenvs/phase/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
max_length = 100
FEATURES_TINY = "data/phase/ml_features_tiny.csv"
FEATURES = "data/phase/ml_features.csv"
dataset_train = "data/phase/ml_features_train.csv"
dataset_test = "data/phase/ml_features_test.csv"
STA = "URZ"
phases = ["regP", "regS", "tele", "N"]
channels = ["BHE", "BHZ", "BHN"]
classifiers = ["Neural Network", "SVM", "XGBoost", "gcForest"]
validation_split = 0.1
seed = 10
file_stack1 = "results/phase_stack_1.hdf5"

# parameters for NN:
batch_size = 1024
epochs = 10
dropout = 0.2
layers = [32, 32]
phase_length = {"URZ": {"regP": 6840, "regS": 6840, "tele": 6840, "N": 20520}}
model_file_path_nn = "results/phase_nn.hdf5"
verbose = 0
cross_validation = False

In [3]:
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__)

Python version 3.5.2 (default, Nov 23 2017, 16:37:01) 
[GCC 5.4.0 20160609]
Pandas version 0.22.0
Matplotlib version 2.1.2


In [4]:
def print_report(model, x_test, y_test):
    print("Best parameters set found on development set:")
    print()
    print(model.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = model.cv_results_['mean_test_score']
    stds = model.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, model.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()
    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, model.predict(x_test)
    print(classification_report(y_true, y_pred))
    print()

In [5]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    column_width = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * column_width
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(column_width) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(column_width) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(column_width) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

In [6]:
def stack1_init(h5f, classifiers):
    try:
        dset_classifier = h5f['/classifier']
    except KeyError:
        dset_classifier = h5f.create_dataset("classifier", data = [c.encode() for c in classifiers])
        
def stack1_save(h5f, arid, index, data):
    dset_classifier = h5f['/classifier']
    try:
        dset_probability = h5f["/probability/{}".format(arid)]
    except KeyError:
        dset_probability = h5f.create_dataset("/probability/{}".format(arid), (len(dset_classifier), 4))
    dset_probability[index] = data

In [7]:
def sparsify(y, n_classes=4):
        'Returns labels in binary NumPy array'
        return np.array([[1 if y[i] == j else 0 for j in range(n_classes)]
                         for i in range(len(y))])

In [8]:
class NN():
    def __init__(self):
        self.model = None
       
    @staticmethod
    def create_model(layers, dropout=0.1):
        # create model
        model = Sequential()
        model.add(Dense(layers[0], input_shape=(1, 16), activation='relu'))
        model.add(Dropout(dropout))
        for units in layers[1:]:
            model.add(Dense(units, activation='relu'))
            model.add(Dropout(dropout))
        model.add(Dense(4, activation='softmax'))

        # Compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model
    
    def fit(self, x_train, y_train):
        x_train = np.expand_dims(x_train, axis=1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
        y_train = sparsify(y_train)
        y_train = np.expand_dims(y_train, axis=1)   
        tensorboard = TensorBoard(log_dir='graph', histogram_freq=0, write_graph=True, write_images=True)
        checkpoint = ModelCheckpoint(model_file_path_nn, monitor='acc', verbose=verbose,
                                     save_best_only=True, mode='max')
        self.model = NN.create_model(layers=layers, dropout=dropout)            
        history = self.model.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=epochs, verbose=verbose,
                  validation_split=0.1, callbacks=[checkpoint, tensorboard])

        print("Max of acc: {}, val_acc: {}".
              format(max(history.history["acc"]), max(history.history["val_acc"])))
        print("Min of loss: {}, val_loss: {}".
              format(min(history.history["loss"]), min(history.history["val_loss"])))
    
    def predict(self, x_test, y_test=None):
        x_test = np.expand_dims(x_test, axis=1) 
        if y_test is not None:
            y_test = sparsify(y_test)
            y_test = np.expand_dims(y_test, axis=1) 
            score = self.model.evaluate(x_test, y_test, verbose=0)
            print("predict:")
            print("Accuracy: {}".format(score[1]*100))
        probability = self.model.predict(x_test, verbose=0)
        return probability  
        
    def load(self, model_file_path):
        self.model = load_model(model_file_path)
        
    def save(self, model_file_path):        
        # save model to file
        self.model.save(model_file_path)
        

In [9]:
class SVM():
    def __init__(self):
        self.model = None
     
    @staticmethod
    def create_model():
        params_grid = [
            #{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
            #{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
            {'C': [1000], 'gamma': [0.001], 'kernel': ['rbf'], 'probability': [True]}
        ]

        model = GridSearchCV(svm.SVC(), params_grid, cv=5, scoring='accuracy', n_jobs=-1)
        return model
    
    def fit(self, x_train, y_train):
        self.model = SVM.create_model()
        print(self.model)
        self.model.fit(x_train, y_train)

    def predict(self, x_test, y_test=None):
        probability = self.model.predict_proba(x_test)
        if y_test is not None:
            y_pred = self.model.predict(x_test)
            prediction = [np.round(value) for value in y_pred]
            accuracy = accuracy_score(y_test, prediction)
            print("Accuracy: %.2f%%" % (accuracy * 100.0))
        return probability
        
    def load(self, model_file_path):
        self.model = joblib.load(model_file_path)
        
    def save(self, model_file_path):     
        # save model to file
        joblib.dump(self.model, model_file_path)
        

In [10]:
class XGBoost():
    def __init__(self):
        self.model = None
    
    @staticmethod
    def create_model():
        seed = 10
        cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
        # set xgboost params
        params_grid = {
            #'max_depth': [5, 6, 7, 8],
            #'n_estimators': [i for i in range(88, 92, 1)],
            #'learning_rate': np.linspace(0.1, 1, 20),
            'max_depth': [6],
            'n_estimators': [i for i in range(90, 91, 1)],
            'learning_rate': np.linspace(0.1, 1, 2),
        }

        params_fixed = {
            'objective': 'multi:softprob',
            'silent': 1,
            'n_jobs': -1,
            'verbose_eval': True
        }

        num_round = 30  # the number of training iterations

        model = GridSearchCV(
            estimator=xgb.XGBClassifier(**params_fixed, seed=seed),
            param_grid=params_grid,
            cv=cv,
            scoring='accuracy'
        )
        return model
    
    def fit(self, x_train, y_train):
        self.model = XGBoost.create_model()
        print(self.model)
        sme = SMOTEENN(random_state=42)
        x_train_res, y_train_res = sme.fit_sample(x_train, y_train)
        self.model.fit(x_train_res, y_train_res)
        
    def predict(self, x_test, y_test=None):
        probability = self.model.predict_proba(x_test)
        if y_test is not None:
            y_pred = self.model.predict(x_test)
            prediction = [np.round(value) for value in y_pred]
            # evaluate predictions
            accuracy = accuracy_score(y_test, prediction)
            print("Accuracy: {}".format(accuracy * 100.0))
        return probability
        
    def load(self, model_file_path):
        self.model = joblib.load(model_file_path)
        
    def save(self, model_file_path):     
        # save model to file
        joblib.dump(self.model, model_file_path)

In [11]:
class GCForest():
    def __init__(self):
        self.model = None
     
    @staticmethod
    def create_model():
        config = {
              "cascade": {
                  "random_state": 0,
                  "max_layers": 100,
                  "early_stopping_rounds": 3,
                  "n_classes": 4,
                  "estimators": [
                      {"n_folds":5,"type":"RandomForestClassifier","n_estimators":10,"max_depth":None,"n_jobs":-1},
                      #{"n_folds":5,"type":"XGBClassifier","n_estimators":10,"max_depth":5,
                      #     "objective":"multi:softprob", "silent":True, "nthread":-1, 
                      #     "learning_rate":0.1},
                      {"n_folds":5,"type":"ExtraTreesClassifier","n_estimators":10,"max_depth":None,"n_jobs":-1},
                      {"n_folds":5,"type":"LogisticRegression"}
                  ]
              }
            }

        model = gcforest.gcforest.GCForest(config)
        return model
    
    def fit(self, x_train, y_train):
        self.model = GCForest.create_model()
        print(self.model)
        self.model.fit_transform(x_train, y_train)        
        
    def predict(self, x_test, y_test=None):
        probability = self.model.predict_proba(x_test)
        if y_test is not None:
            y_pred = self.model.predict(x_test)
            prediction = [np.round(value) for value in y_pred]
            accuracy = accuracy_score(y_test, prediction)
            print("Accuracy: %.2f%%" % (accuracy * 100.0))
        return probability
        
    def load(self, model_file_path):
        self.model = joblib.load(model_file_path)
        
    def save(self, model_file_path):     
        # save model to file
        joblib.dump(self.model, model_file_path)

In [12]:
# Initialize the stack1 file 
h5f = h5py.File(file_stack1, "w")
stack1_init(h5f, classifiers)
h5f.close()

In [13]:
# load train dataset
pd_train = PhaseFeaturesLoader(filename=dataset_train, validation_split=validation_split,
                         phase_length=phase_length, batch_size=batch_size)

x_train, y_train = pd_train.get_dataset(expand_dim=False, y_onehot=False)

length regP:6840
length regS:6840
length tele:6840
length N:20520


In [14]:
# load test dataset
pd_test = PhaseFeaturesLoader(filename=dataset_test, phase_length=phase_length, batch_size=batch_size)
x_test, y_test = pd_test.get_dataset(expand_dim=False, y_onehot=False)
print(pd_test.get_phase_index(100089180))

length regP:2280
length regS:2280
length tele:2280
length N:6840
0


In [15]:
print(x_train.shape)
print(x_test.shape)

(41040, 16)
(13680, 16)


In [16]:
classifiers = ["NN", "SVM", "XGBoost", "GCForest"]
classifier_index = {classifier: i for i, classifier in enumerate(classifiers)}
functions = globals().copy()
classifier_class = {c: functions.get(c) for c in classifiers}
print(classifier_class)
print(classifier_index)

{'XGBoost': <class '__main__.XGBoost'>, 'NN': <class '__main__.NN'>, 'SVM': <class '__main__.SVM'>, 'GCForest': <class '__main__.GCForest'>}
{'XGBoost': 2, 'NN': 0, 'SVM': 1, 'GCForest': 3}


In [17]:
arids = pd_test.get_ids()

In [18]:
for classifier in classifiers:
    model = classifier_class[classifier]()
    model.fit(x_train, y_train)
    model.save("results/phase_train_{}.mdl".format(classifier.lower()))
    probability = model.predict(x_test, y_test)
    for i, arid in enumerate(arids):
        print("arid: {}, proba: {}, y: {}".format(arid, probability[i], y_test[i]))
        if i >= 10:
            break

Max of acc: 0.6556746805672111, val_acc: 0.699317738791423
Min of loss: 0.812684848914292, val_loss: 0.7458223198589525
predict:
Accuracy: 70.05116959064327
arid: 69604157, proba: [[0.6791008  0.00223833 0.2684865  0.0501744 ]], y: 0
arid: 61608769, proba: [[0.02106345 0.19713983 0.07202364 0.70977306]], y: 1
arid: 35743138, proba: [[0.03073326 0.05043396 0.05203848 0.8667943 ]], y: 3
arid: 84668653, proba: [[0.71791524 0.00149557 0.2288035  0.05178571]], y: 0
arid: 99582459, proba: [[0.41767588 0.00899581 0.27152824 0.30180007]], y: 3
arid: 15301629, proba: [[0.0182363  0.39722526 0.09704833 0.48749018]], y: 3
arid: 28075686, proba: [[0.01663931 0.5210031  0.10620998 0.3561475 ]], y: 3
arid: 81864273, proba: [[0.02709439 0.57804734 0.18832013 0.20653816]], y: 1
arid: 121436627, proba: [[0.01523543 0.01778264 0.02845851 0.93852335]], y: 3
arid: 60381321, proba: [[0.06511082 0.33958364 0.0727029  0.5226026 ]], y: 2
arid: 47370982, proba: [[0.01022348 0.54931104 0.09300622 0.34745923]], 

[ 2018-03-28 20:07:41,168][cascade_classifier.fit_transform] X_groups_train.shape=[(41040, 16)],y_train.shape=(41040,),X_groups_test.shape=no_test,y_test.shape=no_test
[ 2018-03-28 20:07:41,169][cascade_classifier.fit_transform] group_dims=[16]
[ 2018-03-28 20:07:41,170][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-03-28 20:07:41,170][cascade_classifier.fit_transform] group_ends=[16]
[ 2018-03-28 20:07:41,171][cascade_classifier.fit_transform] X_train.shape=(41040, 16),X_test.shape=(0, 16)
[ 2018-03-28 20:07:41,173][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(41040, 16), X_cur_test.shape=(0, 16)


Accuracy: 72.64619883040936
arid: 69604157, proba: [9.9727947e-01 5.1334776e-08 2.7149587e-03 5.5050232e-06], y: 0
arid: 61608769, proba: [7.7785910e-05 8.9660579e-01 5.0218686e-02 5.3097706e-02], y: 1
arid: 35743138, proba: [5.6787278e-04 8.2083084e-03 7.3777507e-03 9.8384607e-01], y: 3
arid: 84668653, proba: [9.9948138e-01 4.8130399e-08 5.1849242e-04 1.3714946e-07], y: 0
arid: 99582459, proba: [9.9042106e-01 1.6049256e-05 7.7369376e-03 1.8259495e-03], y: 3
arid: 15301629, proba: [4.5310586e-05 9.7115821e-01 1.6710233e-02 1.2086293e-02], y: 3
arid: 28075686, proba: [8.3121187e-05 7.7143037e-01 2.1689783e-01 1.1588650e-02], y: 3
arid: 81864273, proba: [3.9291240e-07 1.3285497e-01 8.6706066e-01 8.3938903e-05], y: 1
arid: 121436627, proba: [2.0858168e-05 3.0490404e-05 2.1999513e-04 9.9972862e-01], y: 3
arid: 60381321, proba: [1.9034736e-02 9.5777225e-01 2.3109468e-02 8.3532796e-05], y: 2
arid: 47370982, proba: [1.2950872e-07 9.9619019e-01 3.8040639e-03 5.6259655e-06], y: 1


[ 2018-03-28 20:07:41,530][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 5_folds.train_0.predict)=73.84%
[ 2018-03-28 20:07:41,853][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 5_folds.train_1.predict)=74.13%
[ 2018-03-28 20:07:42,174][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 5_folds.train_2.predict)=73.93%
[ 2018-03-28 20:07:42,495][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 5_folds.train_3.predict)=74.63%
[ 2018-03-28 20:07:42,815][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 5_folds.train_4.predict)=74.43%
[ 2018-03-28 20:07:42,817][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 5_folds.train_cv.predict)=74.19%
[ 2018-03-28 20:07:43,050][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_1 - 5_folds.train_0.predict)=74.29%
[ 2018-03-28 20:07:43,270][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_1 - 5_folds.train_1.predict)=74.63%
[ 2018-

[ 2018-03-28 20:08:23,053][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 5_folds.train_0.predict)=74.91%
[ 2018-03-28 20:08:23,274][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 5_folds.train_1.predict)=73.56%
[ 2018-03-28 20:08:23,497][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 5_folds.train_2.predict)=74.35%
[ 2018-03-28 20:08:23,718][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 5_folds.train_3.predict)=75.04%
[ 2018-03-28 20:08:23,941][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 5_folds.train_4.predict)=73.96%
[ 2018-03-28 20:08:23,943][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 5_folds.train_cv.predict)=74.37%
[ 2018-03-28 20:08:26,101][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_2 - 5_folds.train_0.predict)=76.52%
[ 2018-03-28 20:08:28,062][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_2 - 5_folds.train_1.predict)=75.26%
[ 2018-

Accuracy: 77.19%
arid: 69604157, proba: [7.4090809e-01 3.4841083e-04 1.7340475e-01 8.5338831e-02], y: 0
arid: 61608769, proba: [0.00152014 0.37141058 0.08975059 0.5373187 ], y: 1
arid: 35743138, proba: [0.01188111 0.02203865 0.04967971 0.9164005 ], y: 3
arid: 84668653, proba: [8.9830559e-01 2.6878130e-04 7.4064136e-02 2.7361521e-02], y: 0
arid: 99582459, proba: [0.17047971 0.00154391 0.08510912 0.74286723], y: 3
arid: 15301629, proba: [0.00201666 0.42962763 0.1686209  0.39973482], y: 3
arid: 28075686, proba: [0.00193531 0.57114047 0.2833099  0.14361435], y: 3
arid: 81864273, proba: [0.00122345 0.5824758  0.24257678 0.173724  ], y: 1
arid: 121436627, proba: [0.00497894 0.01688245 0.00762629 0.97051233], y: 3
arid: 60381321, proba: [0.02419232 0.50248176 0.1295938  0.34373203], y: 2
arid: 47370982, proba: [4.0298238e-04 7.6103991e-01 9.6017897e-02 1.4253926e-01], y: 1


In [19]:
h5f = h5py.File(file_stack1, "r+")
for i, classifier in enumerate(classifiers):
    model = classifier_class[classifier]()
    model.load("results/phase_train_{}.mdl".format(classifier.lower()))
    probability = model.predict(x_test, y_test)
    for j, arid in enumerate(arids):
        stack1_save(h5f, arid, i, probability[j, :])
h5f.close()

predict:
Accuracy: 70.05116959064327
Accuracy: 73.87%
Accuracy: 72.64619883040936


[ 2018-03-28 20:31:33,651][cascade_classifier.transform] X_groups_test.shape=[(13680, 16)]
[ 2018-03-28 20:31:33,652][cascade_classifier.transform] group_dims=[16]
[ 2018-03-28 20:31:33,653][cascade_classifier.transform] X_test.shape=(13680, 16)
[ 2018-03-28 20:31:33,654][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(13680, 16)
[ 2018-03-28 20:31:34,699][cascade_classifier.transform] [layer=1] look_indexs=[0], X_cur_test.shape=(13680, 28)
[ 2018-03-28 20:31:35,749][cascade_classifier.transform] X_groups_test.shape=[(13680, 16)]
[ 2018-03-28 20:31:35,750][cascade_classifier.transform] group_dims=[16]
[ 2018-03-28 20:31:35,751][cascade_classifier.transform] X_test.shape=(13680, 16)
[ 2018-03-28 20:31:35,751][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(13680, 16)
[ 2018-03-28 20:31:36,796][cascade_classifier.transform] [layer=1] look_indexs=[0], X_cur_test.shape=(13680, 28)


Accuracy: 77.19%
